# SEIR model for simulating spread of COVID19 in Nepal's provinces

COVIDSIM.TEAM stuff here

# SEIR Model

$SEIR$ model is a [compartmental disease model](https://en.wikipedia.org/wiki/Compartmental_models_in_epidemiology). It's goal is to simulate spread of a disease by dividing the population in the following compartments: `Susceptible`, `Exposed`, `Infectious` and `Recovered`. Our model uses a slight variation of the $SEIR$ model. Instead of just one compartment for the infectious population, the model uses two compartments: Documented Infectious ($I^r$) and Undocumented Infectious ($I^u$). Following is a brief explanation of each compartment represented in our model.

- $S$: The number of people susceptible to the disease in each province. This effectivly is the entire population of the province minus the already exposed or infectectious.
- $E$: The number of people exposed to the disease in each province. The individuals in this compartment are hosting the disease in their bodies but are not infectious yet.
- $I^r$: The number of people who are infectious and the case is recorded. These individuals are placed in isolation or are in the hospital undergoing medical treatment.
- $I^u$: The number of people who are infectious, but the case is not documented. These individuals' cases are not document for some reason. Some potential scenarios are: (i) They are `asymptomatic`. It is likely that even they do not know they have the disease, (ii) They are keeping their infectiousness secret because of personal/societal reasons.
- $R$: The number of people who have successfully recovered, or have died. The recovered individuals are also in the susceptible compartment as cases of reinfection has already been observed.

The $SEIR$ model uses data, like the number of documented infectious ($I^r$) in each province, the number of people moving between provinces, the total population of each province, etc. to train its parameters. We can use those parameters to simulate daily number of population in each compartment for each province.

# Data

A model needs data to train its parameters. Obtaining the data regarding numbers of documented infectious, arrivals, quarantined, isolated, etc. was quite straightforward. Estimating latest population and figuring out mobility data was quite complex and a lot of appromixations had to be performed. 

### Covid 19 data

For data regarding the virus, we downloaded `Situation Report` PDF files from [National Disaster Risk Reduction and Management Authority (NDRRMA)'s Nepal COVID-19 Dashboard](https://covid19.ndrrma.gov.np/). We extracted all the data available in province level aggregation from those PDF files. We had data for number of arrivals, quarantined and isolated individuals, deaths, etc. between $29^{th} March, 2020$ and $25^{th} September, 2020$ ($\approx 6$ months).

### Mobility Data

For mobility data, we used a very small, aggregated dataset that `NCELL` kindly provided. The data contained estimations for the number of people departing out of, or arriving in to a district for every district. Our model needed the mobility data to be in the format of number of people moving from each province to another ($M_{ij}$ - number of people moving from $i$ province to $j$ province, for every $i$ and $j$). There were other issues with data compatibility. For instance, (i) data for few districts were aggregated, (ii) there were district spelling mismatches, (iii) data for districts Dolpa, Humla and Mustang were missing altogether, (iV) there were some missing values and most importantly **the data spanned between $31^{st}$ May, 2020 and $16^{th}$ June, 2020** while rest of our data were between $29^{th}$ March and $25^{th}$ September, 2020. 

  
We resolved simpler issues with ease, however, approximating mobility data from the format of `incoming count` and `outgoing_count` for every district, to number of people moving between provinces $i$ and $j$ was relatively complicated. Moreover, we had to extrapolate the mobility data to correspond with the full time frame of rest of the data. A more detailed documentation on issues with data and our workaround for those can be found [here](https://github.com/covidsimteam/py-core/blob/SEIR-intercity/SEIR-intercity/data_wrangling_NBs/mobility_data_cleaning_documentation.md).
  
All the work put into cleaning these mobility data can be found in [this Jupyter Notebook](https://github.com/covidsimteam/py-core/blob/SEIR-intercity/SEIR-intercity/data_wrangling_NBs/ncell_data_cleaner.ipynb). The notebook contains Python codes for data engineering process and is thoroughly documented.

### Population Data

The model also needed population data for each province to calculate the number of susceptible individuals. The only population data we could find dated back to 2011, when the last census was held. For more recent population data we trained a machine learning model to predict the population based on satellite imageries. Although, accuracy of the model cannot be measured until the next census, the total population predicted by the model closely matched the forecast of [Central Bureau of Statistics, Nepal](https://censusnepal.cbs.gov.np/Home/Index/EN).

# Model Dynamics

### Parameters of the model

The model has the following inferrable global, time-invariant parameters:

* $\beta$: The transmission rate due to documented-infectious individuals. This is the rate at which the disease is transmitted from documented individuals.
* $\mu$: The relative transmission rate due to undocumented-infectious individuals. This will act through the product $\mu \beta$ and reduce the effect of $\beta$. This rate reduces the effect of $\beta$ because we assume undocumented cases are mostly because of asymptomatic nature of the individual and are likely to be less infectious.
* $\theta$: The intercity mobility factor. This is a factor greater than 1 and compensates for under-reporting in movement of individuals.
* $Z$: Time period between contracting the virus and eventually being infectious.
* $\alpha$: Fraction of infectious who are actually documented.
* $D$: Time period between being infectious and recovery.

### Model Equations

$\frac{dS_i}{dt} = -\beta \frac{S_i I_i^r}{N_i} - \mu \beta \frac{S_i I_i^u}{N_i} + \theta \sum_k \frac{M_{ij} S_j}{N_j - I_j^r} - + \theta \sum_k \frac{M_{ji} S_j}{N_i - I_i^r}$

This equation represents the change in number of susceptible individuals in province $i$ ($S_i$). As can be seen, new infections reduce the number of susceptible because infected population are moved to infectious compartment. Similarly, mobility to province $i$ increases the number of susceptible individulas in province $i$ and mobility from province $i$ reduces it. Mobility term $M_{ij}$ represent number of people moving from province $i$ to province $j$.
  
  
$\frac{dE_i}{dt} = \beta \frac{S_i I_i^r}{N_i} + \mu \beta \frac{S_i I_i^u}{N_i} -\frac{E_i}{Z} + \theta \sum_k \frac{M_{ij} E_j}{N_j - I_j^r} - + \theta \sum_k \frac{M_{ji} E_j}{N_i - I_i^r}$

This equation represents the change in number of individuals exposed to the virus in province $i$ ($E_i$). Any kind of infectious individual will contribute to growth of number of exposed individuals. Mobility has similar effect as in previous equation. The term $-\frac{E_i}{Z}$ represents the number of individuals moving from exposed compartment to one of infectious compartments.
  
  
$\frac{dI^r_i}{dt} = \alpha \frac{E_i}{Z} - \frac{I_i^r}{D}$
 
This equation represents the change in number of documented infectious individuals. In this equation, the movement of individuals from exposed compartment to infectious compartment is multiplied by $\alpha$ which represents the fraction of infectious population that are actually documented. The term $-\frac{I_i^r}{D}$ represents number of individuals moving from infectious compartment to the recovered compartment.
  
  
$\frac{dI^u_i}{dt} = (1 - \alpha) \frac{E_i}{Z} - \frac{I_i^u}{D} + \theta \sum_k \frac{M_{ij} I_j^u}{N_j - I_j^r} - + \theta \sum_k \frac{M_{ji} I^u_j}{N_i - I_i^r}$

This equation represents the change in number of undocumented infectious individuals. Mobility has the same effect in the euqation as in earlier equations. Rest of the equation is similar to the previous one, except this time we multiply the movement of individuals from exposed  compartment to infectious compartment by the fraction of infectious who are not documented.
  
  
$N_i = N_i + \theta \sum_j M_{ij} - \theta \sum_j M_{ji}$

This equation represents the total population of province $i$. 

### Training the model

The model was trained by running an Ensemble-adjusted Kalman Filter (EAKF) forward in time. We made extensive use of [already existing codes](https://www.tensorflow.org/probability/examples/Undocumented_Infection_and_the_Dissemination_of_SARS-CoV2) from authors at Tensorflow. This code is an implementation of the paper by Li et al. We used these codes becuase it makes use of all available cores in the computation device of choice, resulting in great efficiency.

# Results

Follwing plot show the performance of this $SEIR$ model.

<img src="seir-result.png">